# Check synchronization of local and cloud files and directories

In [ ]:
import os

instance_name = f"test-sqlite-sync"
!lamin load {instance_name}
!yes | lamin delete {instance_name}

In [ ]:
from lamindb_setup import init, settings
import os

In [ ]:
init(
    storage=f"s3://lamindb-ci/{instance_name}",
    name=instance_name,
)

Test sync of general files and directories

In [ ]:
dir_sync = settings.storage.root / "dir_sync"
if dir_sync.is_dir():
    dir_sync.rmdir()

(dir_sync / "file1").touch()
(dir_sync / "file2").touch()

assert dir_sync.is_dir()

In [ ]:
dir_sync_local = settings.storage.cloud_to_local(dir_sync)

In [ ]:
assert dir_sync_local.is_dir()
assert len(list(dir_sync_local.iterdir())) == 2

In [ ]:
for file in ("file1", "file2"):
    assert (dir_sync_local / file).stat().st_mtime == (
        dir_sync / file
    ).modified.timestamp()

In [ ]:
local_file = dir_sync_local / "file1"
local_file.unlink()
assert not local_file.exists()
assert len(list(dir_sync_local.iterdir())) == 1

In [ ]:
dir_sync_local = settings.storage.cloud_to_local(dir_sync)
assert local_file.exists()
assert len(list(dir_sync_local.iterdir())) == 2

In [ ]:
for file in ("file1", "file2"):
    cloud_file = dir_sync / file
    local_file = dir_sync_local / file

    cloud_mtime = cloud_file.modified.timestamp()
    os.utime(local_file, times=(cloud_mtime - 1, cloud_mtime - 1))

    assert local_file.stat().st_mtime < cloud_mtime

In [ ]:
dir_sync_local = settings.storage.cloud_to_local(dir_sync)

for file in ("file1", "file2"):
    assert (dir_sync_local / file).stat().st_mtime == (
        dir_sync / file
    ).modified.timestamp()

In [ ]:
(dir_sync_local / "file1").unlink()

local_file_new = dir_sync_local / "file3"
local_file_new.touch()
assert len(list(dir_sync_local.iterdir())) == 2

In [ ]:
dir_sync_local = settings.storage.cloud_to_local(dir_sync)

assert len(list(dir_sync_local.iterdir())) == 2
assert not local_file_new.exists()

In [ ]:
dir_sync.rmdir()

for file in dir_sync_local.iterdir():
    file.unlink()
dir_sync_local.rmdir()

Get the paths to the cloud and local sqlite databases.

In [ ]:
sqlite_file = settings.instance._sqlite_file

In [ ]:
sqlite_file

Remote SQLite file does exists upon instance init:

In [ ]:
assert settings.instance._sqlite_file.exists()

Now mimic a new user who loads the instance (this runs 4s):

In [ ]:
settings.instance._update_local_sqlite_file()

Get the mere filepath of the local file, without any update:

In [ ]:
cache_file = settings.instance.storage.cloud_to_local_no_update(sqlite_file)

In [ ]:
cache_file

Delete the local sqlite file:

In [ ]:
cache_file.unlink()

In [ ]:
assert not cache_file.exists()

Update the local version of the sqlite file:

In [ ]:
settings.instance._update_local_sqlite_file()

In [ ]:
assert cache_file.exists()

If the local sqlite database is older than the cloud one, the cloud database replaces the local sqlite database file.

In [ ]:
cloud_mtime = sqlite_file.modified.timestamp()

In [ ]:
cloud_mtime

In [ ]:
os.utime(cache_file, times=(cloud_mtime - 1, cloud_mtime - 1))

In [ ]:
assert cache_file.stat().st_mtime < sqlite_file.modified.timestamp()

In [ ]:
settings.instance._update_local_sqlite_file()

In [ ]:
assert cache_file.stat().st_mtime == sqlite_file.modified.timestamp()

In [ ]:
!yes | lamin delete {instance_name}